# Import Packages

In [4]:
import time

import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import nengo
import numpy as np
import PIL.Image

from collections import deque

from alive_progress import alive_bar
# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib
from dm_control import composer, mjcf, mujoco
from dm_control.utils import inverse_kinematics as ik
from IPython.display import HTML

%env MUJOCO_GL=egl
%matplotlib widget

env: MUJOCO_GL=egl


# Define Utilities and Parameters

In [5]:
# Rendering parameters
dpi = 100
framerate = 60 # (Hz)
width, height = 720, 480
sensor_shape = (10, 10)

# IK solver parameters
_MAX_STEPS = 50
_TOL = 1e-12
_TIME_STEP = 2e-3


def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())


def get_jacobian(physics, site_name):
    phys = mujoco.Physics.from_model(physics.model)
    jac_pos = np.zeros((3, phys.model.nv))
    jac_rot = np.zeros((3, phys.model.nv))
    mjlib.mj_jacSite(
        phys.model.ptr,
        phys.data.ptr,
        jac_pos,
        jac_rot,
        phys.model.name2id(site_name, 'site'))
    
    return jac_pos, jac_rot

# Run Simulations

In [125]:
# Load scene
scene_xml = 'models/scene.xml'
physics = mujoco.Physics.from_xml_path(scene_xml)

# Define simulation variables
site_name = 'attachment_site'
control_site = physics.data.site(name=site_name)
target_site = physics.data.site(name='reach_site2')
joint_names = ['joint{}'.format(i+1) for i in range(7)]
ctrl = np.zeros(10)
duration = 3.0 # (seconds)
omega = np.pi/2 # Rotator angular velocity
video = []
stream = []

# Simulate, saving video frames
physics.reset(1)
ctrl[:7] = physics.data.qpos[:7]
xpos = control_site.xpos
with alive_bar(int(duration/_TIME_STEP) + 1, ctrl_c=False, force_tty=True, title='Rendering') as bar:
    while physics.data.time < duration:
        if control_site.xpos[2] > target_site.xpos[2] + 0.1:
            xpos[2] -= 0.05
            result = ik.qpos_from_site_pose(
              physics=physics,
              site_name=site_name,
              target_pos=target_site.xpos,
              target_quat=[0, 0, 0, 1],
              joint_names=joint_names,
              tol=_TOL,
              max_steps=_MAX_STEPS,
              inplace=False,
            )
            if result.success:
                ctrl[:7] = result.qpos[:7]
        ctrl[-3:] = np.array([1, 1, 1])*omega*physics.data.time
        physics.set_control(ctrl)
        physics.step()
    
        data = physics.data.sensordata
        stream.append(data[0])
        # Save video frames and sensor data
        if len(video) < physics.data.time * framerate:
            pixels = physics.render(camera_id='prospective', width=width, height=height)
            video.append(pixels.copy())
        
        # Update progress bar
        bar()

# PIL.Image.fromarray(image)
display_video(video, framerate)

Rendering |████████████████████████████████████████| 1501/1501 [100%] in 8.0s (187.72/s)                                


In [ ]:
plt.plot(stream)
plt.show()

# Modify Taxel Sites in The Scene XML File
### [Mujoco XML Reference](https://mujoco.readthedocs.io/en/latest/XMLreference.html?highlight=site#body-site)
------------------------------------------
## Taxel Sites
* site_type: [sphere, capsule, ellipsoid, cylinder, box], “sphere”
* dx: the distance between each site
* offset: uniform distance to shift all sites
## Composite
* _M: (x/y-count, z-count)
* geom_type: [particle, grid, cable, rope, loop, cloth, box, cylinder, ellipsoid], required

In [124]:
from xml.etree import ElementTree as et


# Parse the XML
robot_xml = 'models/panda_nohand.xml'
tree = et.parse(robot_xml)
root = tree.getroot()

# Define taxel sites to place touch sensors
_N = 10
site_type = 'capsule'
dx = 3e-3
offset = dx*(_N - 1)/2.0

for body in root.findall('.//body'):
    if body.get('name') == 'taxel_sites':
        sites = list(body.iter('site'))
        for i, site in enumerate(sites):
            site.attrib['type'] = site_type
            site.attrib['size'] = '1e-3 .002'
            site.attrib['pos'] = '{:.5f} -.003 {:.5f}'.format(dx*(i//_N)-offset, dx*(i%_N)-offset) # x-z-y axis
            site.attrib['euler'] = '1.570796 0 0'
            
# Define composite to simulate softbody
_M = (11, 2)
geom_type = 'capsule'

for body in root.findall('.//body'):
    if body.get('name') == 'sensor_pad':
        body.attrib['pos'] = '0 -0.0045 0'
composite = root.find('.//composite')
composite.attrib['count'] = '{} {} {}'.format(_M[0], _M[0], _M[1])
composite.attrib['spacing'] = str(.03/(_M[0] - 1))
geom = composite.find('geom')
geom.attrib['type'] = geom_type
geom.attrib['size'] = '.001 .002'
joint = composite.find('joint')
joint.attrib['solreffix'] = '.01 1'
joint.attrib['solimpfix'] = '0 .1 .01'
tree.write(robot_xml) # Save robot XML

# Modify the keyframe home key qpos
scene_xml = 'models/scene.xml'
tree = et.parse(scene_xml)
root = tree.getroot()
key = root.find('.//keyframe/key')
key.attrib['qpos'] = "0 0 0 -1.570796 0 1.570796 0 0 0 0 " + ' '.join(['0' for i in range(242)])
tree.write(scene_xml)